# TensorFlow - ML Program


In [9]:
# Example 1
with tf.compat.v1.Session() as ses:

    #costants

    a = tf.constant([1,2])
    b = tf.constant([3,4])

    #Multiply
    result = tf.multiply(a,b)
    # using session

    print(sess.run(result))
    print(result)

[3 8]
Tensor("Mul:0", shape=(2,), dtype=int32)


### Ref: https://www.tensorflow.org/tutorials/quickstart/beginner
This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.24475823, -0.19890061, -0.67481005, -0.21958145,  0.11489888,
        -0.43428394, -0.24984872, -0.5682292 ,  0.5775861 , -0.26433676]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.14077498, 0.0903331 , 0.05612574, 0.08848412, 0.12363127,
        0.07138734, 0.08584607, 0.06243807, 0.19636804, 0.0846113 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.6396346

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4825 - accuracy: 0.8610
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1532 - accuracy: 0.9547
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1104 - accuracy: 0.9654
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0846 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0774 - accuracy: 0.9758


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0737 - accuracy: 0.9772


[0.07369285076856613, 0.9771999716758728]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.11548183e-07, 4.97467179e-09, 3.50532150e-06, 7.28782325e-05,
        4.97650489e-12, 2.75390363e-07, 1.10802765e-13, 9.99921322e-01,
        1.07561128e-07, 1.65880124e-06],
       [2.71735789e-09, 1.33377049e-04, 9.99861121e-01, 5.41055533e-06,
        5.99575501e-16, 1.74293291e-07, 2.86014128e-08, 2.38284029e-13,
        4.55673934e-08, 1.97764101e-14],
       [8.58958948e-09, 9.99482214e-01, 4.87843172e-05, 2.25714757e-06,
        1.00361598e-04, 5.95439678e-06, 3.78748769e-06, 3.07752081e-04,
        4.57943315e-05, 3.24935058e-06],
       [9.99921322e-01, 1.34286859e-09, 1.42423751e-05, 2.96841787e-08,
        2.97537301e-08, 2.67447854e-06, 3.64606494e-05, 6.92822323e-06,
        2.45295695e-08, 1.84794208e-05],
       [2.76027276e-05, 1.01314046e-09, 9.64628725e-07, 1.08510001e-08,
        9.94548261e-01, 5.64389886e-08, 9.48100194e-08, 3.32594645e-04,
        2.45312748e-07, 5.09021711e-03]], dtype=float32)>